In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [1]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band1
  atmpro = mls
  band = 1
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0, 0]
  ng_refs = [3, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (30, 250), (300, 250)]
  tsfc = 294
  vmax = 340
  vmin = 20
  w_diffuse = [(1.66, 1.66, 1.66), (1.66, 1.66, 1.66), (1.66, 1.66, 1.5, 1.5, 1.5, 1.5)]
  wgt = [(0.2, 0.2, 0.2), (0.2, 0.2, 0.2), (0.2, 0.2, 0.65, 0.65, 0.65, 0.65)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  6.011740                  NaN       NaN
0.000312      NaN       NaN                  1.0  3.452597
0.000750      2.0  4.585616                  2.0  3.249962
0.001052      3.0  4.307809                  3.0  3.295524
0.001476      4.0  3.987559                  4.0  3.279020
0.002070      5.0  3.639014                  5.0  3.181410
0.002904      6.0  3.279372                  6.0  2.992595
0.004074      7.0  2.921127                  7.0  2.712360
0.005714      8.0  2.572211                  8.0  2.368199
0.008015      9.0  2.237861                  9.0  2.009496
0.011243     10.0  1.923741                 10.0  1.694401
0.015771     11.0  1.635788                 11.0  1.460934
0.022122     12.0  1.380278                 12.0  1.296986
0.031031     13.0  1.165474                 13.0  1.162094
0.043528     14.0  0.973910                 14.0  1.004653
0.061057     15.0  0.795869                 15.0  0.814592
0.085645     16.0  0.657084                 16.0  0.642948
0.120136     17.0  0.563709                 17.0  0.528796
0.168516     18.0  0.505366                 18.0  0.477355
0.236378     19.0  0.476648                 19.0  0.466735
0.331549     20.0  0.476110                 20.0  0.471967
0.465100     21.0  0.511318                 21.0  0.496357
0.652400     22.0  0.518116                 22.0  0.496195
0.915100     23.0  0.478928                 23.0  0.475617
1.283650     24.0  0.444843                 24.0  0.470742
1.800600     25.0  0.418484                 25.0  0.457568
2.525700     26.0  0.391751                 26.0  0.419543
3.542800     27.0  0.361303                 27.0  0.377826
4.969550     28.0  0.335358                 28.0  0.329042
6.970850     29.0  0.321727                 29.0  0.301855
9.778100     30.0  0.310393                 30.0  0.296863
13.715850    31.0  0.295116                 31.0  0.287570
19.239350    32.0  0.275577                 32.0  0.265473
26.987250    33.0  0.264919                 33.0  0.260151
37.855300    34.0  0.265604                 34.0  0.263540
53.100050    35.0  0.264156                 35.0  0.259271
73.887500    36.0  0.261206                 36.0  0.253524
97.662500    37.0  0.257209                 37.0  0.241395
121.437500   38.0  0.250354                 38.0  0.229545
145.212500   39.0  0.319512                 39.0  0.295326
168.987500   40.0  0.371412                 40.0  0.353046
192.762500   41.0  0.410160                 41.0  0.396118
216.537500   42.0  0.410683                 42.0  0.396448
240.312500   43.0  0.400317                 43.0  0.384966
264.087500   44.0  0.385069                 44.0  0.375799
287.862500   45.0  0.354458                 45.0  0.356335
311.637500   46.0  0.342206                 46.0  0.352379
335.412500   47.0  0.377542                 47.0  0.392239
359.187500   48.0  0.421650                 48.0  0.432640
382.962500   49.0  0.547598                 49.0  0.550032
406.737500   50.0  0.674417                 50.0  0.669400
430.512500   51.0  0.723477                 51.0  0.734126
454.287500   52.0  0.710377                 52.0  0.750129
478.062500   53.0  0.658486                 53.0  0.696826
501.837500   54.0  0.584805                 54.0  0.594319
525.612500   55.0  0.499319                 55.0  0.497928
549.387500   56.0  0.409746                 56.0  0.419432
573.162500   57.0  0.323411                 57.0  0.344199
596.937500   58.0  0.245660                 58.0  0.262366
620.712500   59.0  0.179973                 59.0  0.182207
644.487500   60.0  0.127642                 60.0  0.116475
668.262500   61.0  0.088222                 61.0  0.070001
692.037500   62.0  0.059951                 62.0  0.041231
715.812500   63.0  0.040484                 63.0  0.025112
739.587500   64.0  0.027645                 64.0  0.016637
76

# Fluxes by Layer

CRD                           CLIRAD                \
                         flug       fldg      fnetg       flug          fldg   
pressure     level                                                             
0.000000e+00 1     -31.956085   0.000000 -31.956085        NaN           NaN   
1.000000e-08 1            NaN        NaN        NaN -31.774476  5.809747e-07   
6.244000e-04 2     -31.956149   0.000509 -31.955641 -31.774533  3.132326e-04   
8.759000e-04 3     -31.956176   0.000672 -31.955504 -31.774556  4.329312e-04   
1.228600e-03 4     -31.956210   0.000886 -31.955324 -31.774585  5.996037e-04   
1.723400e-03 5     -31.956251   0.001161 -31.955090 -31.774621  8.281448e-04   
2.417400e-03 6     -31.956303   0.001512 -31.954791 -31.774666  1.134815e-03   
3.390900e-03 7     -31.956365   0.001952 -31.954413 -31.774722  1.535369e-03   
4.756500e-03 8     -31.956441   0.002501 -31.953940 -31.774789  2.041427e-03   
6.672000e-03 9     -31.956531   0.003175 -31.953357 -31.774869  2.659354e-03   
9.358900e-03 10    -31.956638   0.003994 -31.952644 -31.774964  3.394040e-03   
1.312780e-02 11    -31.956763   0.004977 -31.951786 -31.775075  4.260933e-03   
1.841450e-02 12    -31.956905   0.006143 -31.950761 -31.775200  5.301407e-03   
2.583020e-02 13    -31.957061   0.007512 -31.949549 -31.775338  6.578349e-03   
3.623230e-02 14    -31.957219   0.009106 -31.948113 -31.775475  8.147387e-03   
5.082340e-02 15    -31.957365   0.010935 -31.946429 -31.775598  1.000742e-02   
7.129060e-02 16    -31.957507   0.013007 -31.944500 -31.775713  1.209729e-02   
1.000000e-01 17    -31.957650   0.015385 -31.942265 -31.775821  1.439198e-02   
1.402710e-01 18    -31.957782   0.018206 -31.939576 -31.775904  1.699840e-02   
1.967600e-01 19    -31.957871   0.021676 -31.936194 -31.775928  2.021724e-02   
2.759970e-01 20    -31.957852   0.026132 -31.931720 -31.775822  2.449206e-02   
3.871000e-01 21    -31.957563   0.032108 -31.925454 -31.775415  3.029700e-02   
5.431000e-01 22    -31.956403   0.040398 -31.916006 -31.774122  3.817786e-02   
7.617000e-01 23    -31.953962   0.051373 -31.902589 -31.771542  4.844753e-02   
1.068500e+00 24    -31.950595   0.065412 -31.885184 -31.767979  6.217190e-02   
1.498800e+00 25    -31.946436   0.083927 -31.862509 -31.763561  8.175125e-02   
2.102400e+00 26    -31.941330   0.108742 -31.832587 -31.758124  1.090339e-01   
2.949000e+00 27    -31.935230   0.141930 -31.793300 -31.751673  1.446611e-01   
4.136600e+00 28    -31.928698   0.186226 -31.742472 -31.744791  1.909368e-01   
5.802500e+00 29    -31.922487   0.246194 -31.676293 -31.738328  2.494135e-01   
8.139200e+00 30    -31.916263   0.329023 -31.587240 -31.731923  3.265696e-01   
1.141700e+01 31    -31.909880   0.443160 -31.466721 -31.725518  4.354425e-01   
1.601470e+01 32    -31.905028   0.599036 -31.305992 -31.720923  5.874834e-01   
2.246400e+01 33    -31.907134   0.811673 -31.095461 -31.723650  7.930430e-01   
3.151050e+01 34    -31.919629   1.108061 -30.811568 -31.737223  1.085428e+00   
4.420010e+01 35    -31.938689   1.526370 -30.412319 -31.757594  1.501986e+00   
6.200000e+01 36    -31.962525   2.107185 -29.855339 -31.782879  2.074008e+00   
8.577500e+01 37    -31.986509   2.866808 -29.119701 -31.808202  2.813410e+00   
1.095500e+02 38    -32.000745   3.605427 -28.395318 -31.823396  3.508517e+00   
1.333250e+02 39    -32.010448   4.320207 -27.690242 -31.834068  4.165727e+00   
1.571000e+02 40    -32.026163   5.235769 -26.790394 -31.851336  5.014815e+00   
1.808750e+02 41    -32.050749   6.306369 -25.744379 -31.878207  6.036079e+00   
2.046500e+02 42    -32.088263   7.499023 -24.589239 -31.918949  7.192530e+00   
2.284250e+02 43    -32.141282   8.708657 -23.432625 -31.976408  8.366628e+00   
2.522000e+02 44    -32.216420   9.911214 -22.305205 -32.057280  9.531799e+00   
2.759750e+02 45    -32.331992  11.111263 -21.220730 -32.180680  1.071368e+01   
2.997500e+02 46    -32.536817  12.314355 -20.222462 -32.395939  1.193259e+01   
3.235250e+02 47    -32.843

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')